# **Prepare data**

In [ ]:
# Update gdown
!pip install --upgrade --no-cache-dir gdown

# Download the dataset from Google Drive
!gdown --id '16FvOGDl-9cL1nOt0cKKJQvWWysMStTPX&export' --output data.zip
!gdown --id '1ReuU9QWr5Da-VeTOQIFf-WvFZps1PVmW&export' --output yolo_data.zip

# Unzip the dataset.
!unzip -o data.zip
!unzip -o yolo_data.zip
!ls

串流輸出內容已截斷至最後 5000 行。
  inflating: resultTrain/00878.jpg   
  inflating: resultTrain/00879.jpg   
  inflating: resultTrain/00880.jpg   
  inflating: resultTrain/00881.jpg   
  inflating: resultTrain/00883.jpg   
  inflating: resultTrain/00885.jpg   
  inflating: resultTrain/00886.jpg   
  inflating: resultTrain/00887.jpg   
  inflating: resultTrain/00888.jpg   
  inflating: resultTrain/00889.jpg   
  inflating: resultTrain/00892.jpg   
  inflating: resultTrain/00893.jpg   
  inflating: resultTrain/00894.jpg   
  inflating: resultTrain/00895.jpg   
  inflating: resultTrain/00896.jpg   
  inflating: resultTrain/00897.jpg   
  inflating: resultTrain/00898.jpg   
  inflating: resultTrain/00899.jpg   
  inflating: resultTrain/00900.jpg   
  inflating: resultTrain/00901.jpg   
  inflating: resultTrain/00902.jpg   
  inflating: resultTrain/00903.jpg   
  inflating: resultTrain/00904.jpg   
  inflating: resultTrain/00905.jpg   
  inflating: resultTrain/00906.jpg   
  inflating: resultTrain/0090

# **Install package**

In [ ]:
# Import necessary packages
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary
import sys
import os
import csv
import pandas as pd
from PIL import Image
from matplotlib import image
from matplotlib import pyplot
import pickle

# **Data Preprocessing**

In [ ]:
train_label = pd.read_csv('train.csv')
test_label = pd.read_csv('dev.csv')

In [ ]:
train_x_array = []
train_x_yolo_array = []
train_y_array = []
test_x_array = []
test_x_yolo_array = []
test_y_array = []
for index, row in train_label.iterrows():
    if index % 100 == 0:
        print(index)
    train_x_array.append(np.asarray(Image.open(os.path.join('C1-P1_Train', row['image_id'])).resize((224, 224))))
    train_x_yolo_array.append(np.asarray(Image.open(os.path.join('resultTrain', row['image_id'])).resize((224, 224))))
    train_y_array.append(np.asarray(row['label']))
for index, row in test_label.iterrows():
    test_x_array.append(np.asarray(Image.open(os.path.join('C1-P1_Dev', row['image_id'])).resize((224, 224))))
    test_x_yolo_array.append(np.asarray(Image.open(os.path.join('resultDev', row['image_id'])).resize((224, 224))))
    test_y_array.append(np.asarray(row['label']))

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500


In [ ]:
def one_hot(array):
    unique, inverse = np.unique(array, return_inverse=True)
    onehot = np.eye(unique.shape[0])[inverse]
    return onehot

In [ ]:
train_x = np.array(train_x_array)
train_x_yolo = np.array(train_x_yolo_array)
train_y = np.array(train_y_array)
test_x = np.array(test_x_array)
test_x_yolo = np.array(test_x_yolo_array)
test_y = np.array(test_y_array)
train_y = one_hot(train_y)
test_y = one_hot(test_y)

In [ ]:
train_y

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [ ]:
print(train_x.shape, train_x_yolo.shape, train_y.shape, test_x.shape, test_x_yolo.shape, test_y.shape)

(5600, 224, 224, 3) (5600, 224, 224, 3) (5600, 3) (800, 224, 224, 3) (800, 224, 224, 3) (800, 3)


In [ ]:
with open('train_x.npy', 'wb') as f:
    np.save(f, train_x)
with open('train_x_yolo.npy', 'wb') as f:
    np.save(f, train_x_yolo)
with open('train_y.npy', 'wb') as f:
    np.save(f, train_y)
with open('test_x.npy', 'wb') as f:
    np.save(f, test_x)
with open('test_x_yolo.npy', 'wb') as f:
    np.save(f, test_x_yolo)
with open('test_y.npy', 'wb') as f:
    np.save(f, test_y)

In [ ]:
!zip data_np.zip train_x.npy train_x_yolo.npy train_y.npy test_x.npy test_x_yolo.npy test_y.npy

updating: train_x.npy (deflated 18%)
updating: train_x_yolo.npy (deflated 15%)
updating: train_y.npy (deflated 97%)
updating: test_x.npy (deflated 18%)
updating: test_x_yolo.npy (deflated 15%)
updating: test_y.npy (deflated 97%)


# **CNN Model**

In [ ]:
# Set seeds
def set_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seeds(880330)

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
os.makedirs('models', exist_ok=True)

In [ ]:
with open('train_x.npy', 'rb') as f:
    train_x = np.load(f)
with open('train_y.npy', 'rb') as f:
    train_y = np.load(f)
with open('test_x.npy', 'rb') as f:
    test_x = np.load(f)
with open('test_y.npy', 'rb') as f:
    test_y = np.load(f)

In [ ]:
# Define dataset
class train_dataset_mango(Dataset):
    def __init__(self, train_x, train_y, mode='train'):
        # Split data into 8:2
        if mode == 'train':
            indices = [i for i in range(len(train_x)) if i % 5 != 0]
        else:
            indices = [i for i in range(len(train_x)) if i % 5 == 0]
        
        self.xs = torch.FloatTensor(train_x[indices])
        self.ys = torch.FloatTensor(train_y[indices])

        print('Shape of {} data x is {}'.format(mode, self.xs.shape))
        print('Shape of {} data y is {}'.format(mode, self.ys.shape))

        self.dim = self.xs.shape[-1]

    def __getitem__(self, index):
        return self.xs[index], self.ys[index]

    def __len__(self):
        return len(self.xs)

class test_dataset__mango(Dataset):
    def __init__(self, test_x):
        self.xs = torch.FloatTensor(test_x)

        print('Shape of test data x is {}'.format(self.xs.shape))

    def __getitem__(self, index):
        return self.xs[index]

    def __len__(self):
        return len(self.xs)

In [ ]:
# Split data into different dataset
train_dataset = train_dataset_mango(train_x, train_y, mode='train')
dev_dataset = train_dataset_mango(train_x, train_y, mode='dev')
test_dataset = test_dataset__mango(test_x)

Shape of train data x is torch.Size([4480, 224, 224, 3])
Shape of train data y is torch.Size([4480, 3])
Shape of dev data x is torch.Size([1120, 224, 224, 3])
Shape of dev data y is torch.Size([1120, 3])
Shape of test data x is torch.Size([800, 224, 224, 3])


## **Config**

In [ ]:
# Set config
config = {
    'batch': 100,
    'Dropout': 0.25,
    'learning_rate': 0.001,
    'weight_decay': 0.02, 
    'epochs': 100, 
    'n_dim': [32, 32, 32, 64],
    'fc_n_dim': [64*26*26, 1024, 256, 128, 64],
    'early_stop': 10,
    'path': './models/cnn.pth'
}

## **Data Loader**

In [ ]:
# Fit data loader
train_loader = DataLoader(train_dataset, batch_size=config['batch'], shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=config['batch'], shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=config['batch'], shuffle=False)

## **Normalization and Build model**

In [ ]:
# Define CNN model
class CNN(nn.Module):
    def __init__(self, input_dim):
        super(CNN, self).__init__()
        
        # CNN
        self.cnn = nn.Sequential(
            # 3*224*224
            nn.Conv2d(input_dim, config['n_dim'][0], 3, 1),
            nn.BatchNorm2d(config['n_dim'][0]),
            nn.ReLU(),
            # 32*222*222
            nn.Conv2d(config['n_dim'][0], config['n_dim'][1], 3, 1),
            nn.BatchNorm2d(config['n_dim'][1]),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),
            # 32*110*110
            nn.Conv2d(config['n_dim'][1], config['n_dim'][2], 3, 1),  
            nn.BatchNorm2d(config['n_dim'][2]),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),
            # 32*54*54
            nn.Conv2d(config['n_dim'][2], config['n_dim'][3], 3, 1),  
            nn.BatchNorm2d(config['n_dim'][3]),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0)
            # 64*26*26
        )

        # Fully Connected Layer
        self.fc = nn.Sequential(
            nn.Linear(config['fc_n_dim'][0], config['fc_n_dim'][1]),
            nn.Dropout(config['Dropout']),
            nn.ReLU(),
            nn.Linear(config['fc_n_dim'][1], config['fc_n_dim'][2]),        
            nn.Dropout(config['Dropout']),         
            nn.ReLU(),            
            nn.Linear(config['fc_n_dim'][2], config['fc_n_dim'][3]),
            nn.Dropout(config['Dropout']),
            nn.ReLU(),
            nn.Linear(config['fc_n_dim'][3], config['fc_n_dim'][4]),
            nn.Dropout(config['Dropout']),
            nn.ReLU(),
            nn.Linear(config['fc_n_dim'][4], 3)
        )

    def forward(self, x):
        # Change dimension from [data_size, 224, 224, 3] to [data_size, 3, 224, 224]
        x = x.permute(0, 3, 1, 2)
        x = self.cnn(x)

        # Flatten
        x = x.flatten(1)
        x = self.fc(x)
        
        return x

In [ ]:
# Initialize weight
def weight_initialization(layer):
    if isinstance(layer, nn.Linear):
      torch.nn.init.xavier_uniform_(layer.weight)

In [ ]:
# Build model
loss_function = nn.CrossEntropyLoss()
CNN_model = CNN(train_dataset.dim).to(device)
CNN_model.apply(weight_initialization)
optimizer = optim.Adam(CNN_model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])

## **Re-load best weight, and prediction test dataset**

In [ ]:
# Define training
def train(train_loader, dev_loader, model, config, device):
    # settings
    epochs = config['epochs']
    path = config['path']
    
    epoch = 0
    min = sys.maxsize
    early_stop_cnt = 0
    
    while epoch < epochs:
        model.train()
        for i, (x,y) in enumerate(train_loader):
            optimizer.zero_grad()
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss = loss_function(pred, y)
            loss.backward()
            optimizer.step()

        if dev_loader is not None:
            dev_loss = dev(dev_loader, model, device)
            if dev_loss < min:
                torch.save(model, path)
                min = dev_loss
                print('epoch {}: loss = {}'.format(epoch+1, min))
                early_stop_cnt = 0
            else:
                early_stop_cnt += 1
        epoch += 1
        if early_stop_cnt > config['early_stop']:
            break

    print('Finished training after {} epochs'.format(epoch))

# Define validation
def dev(dev_loader, model, device):
    model.eval()
    total_loss = 0
    for x, y in dev_loader:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            pred = model(x)
            ce_loss = loss_function(pred, y)
        total_loss += ce_loss.detach().cpu().item()*len(x)
    
    return total_loss / len(dev_loader.dataset)

In [ ]:
# Training
train(train_loader, dev_loader, CNN_model, config, device)

epoch 1: loss = 1.0517909760986055
epoch 2: loss = 0.9838425708668572
epoch 3: loss = 0.979014512683664
epoch 4: loss = 0.9429380797914096
epoch 5: loss = 0.9226798809000424
epoch 10: loss = 0.9053124572549548
epoch 11: loss = 0.829264552465507
epoch 12: loss = 0.7172596624919346
epoch 21: loss = 0.6800452811377389
epoch 22: loss = 0.6745572090148926
epoch 25: loss = 0.6646365587200437
epoch 30: loss = 0.6531274201614516
epoch 32: loss = 0.6332969729389463
Finished training after 43 epochs


In [ ]:
# Delete the model
del CNN_model

In [ ]:
# Define testing
def test(test_loader, model, device):
    model.eval()
    pred_list = []
    for x in test_loader:
        x = x.to(device)
        with torch.no_grad():
            pred = model(x)
            pred_list.append(pred.detach().cpu())

    pred_list = torch.cat(pred_list, dim=0).numpy()
    print("Done testing")
    return pred_list

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference

In [ ]:
# Testing
CNN_model = torch.load(config['path']).to(device)
test_pred = test(test_loader, CNN_model, device)
res_list = []
for test in test_pred:
    res = softmax(test)
    x = np.zeros(3)
    x[np.argmax(res)] = 1
    res_list.append(x)

Done testing


In [ ]:
total = 0
for i in range(len(res_list)):
    if np.argmax(res_list[i]) == np.argmax(test_y[i]):
        total += 1
print('accuracy', total/len(res_list))

accuracy 0.75125


## **Arch**

In [ ]:
summary(CNN_model, (224, 224, 3))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 222, 222]             896
       BatchNorm2d-2         [-1, 32, 222, 222]              64
              ReLU-3         [-1, 32, 222, 222]               0
            Conv2d-4         [-1, 32, 220, 220]           9,248
       BatchNorm2d-5         [-1, 32, 220, 220]              64
              ReLU-6         [-1, 32, 220, 220]               0
         MaxPool2d-7         [-1, 32, 110, 110]               0
            Conv2d-8         [-1, 32, 108, 108]           9,248
       BatchNorm2d-9         [-1, 32, 108, 108]              64
             ReLU-10         [-1, 32, 108, 108]               0
        MaxPool2d-11           [-1, 32, 54, 54]               0
           Conv2d-12           [-1, 64, 52, 52]          18,496
      BatchNorm2d-13           [-1, 64, 52, 52]             128
             ReLU-14           [-1, 64,

# **CNN Model with YOLO**

In [ ]:
# Set seeds
def set_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seeds(880330)

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
os.makedirs('models', exist_ok=True)

In [ ]:
with open('train_x_yolo.npy', 'rb') as f:
    train_x = np.load(f)
with open('train_y.npy', 'rb') as f:
    train_y = np.load(f)
with open('test_x_yolo.npy', 'rb') as f:
    test_x = np.load(f)
with open('test_y.npy', 'rb') as f:
    test_y = np.load(f)

In [ ]:
# Define dataset
class train_dataset_mango(Dataset):
    def __init__(self, train_x, train_y, mode='train'):
        # Split data into 8:2
        if mode == 'train':
            indices = [i for i in range(len(train_x)) if i % 5 != 0]
        else:
            indices = [i for i in range(len(train_x)) if i % 5 == 0]
        
        self.xs = torch.FloatTensor(train_x[indices])
        self.ys = torch.FloatTensor(train_y[indices])

        print('Shape of {} data x is {}'.format(mode, self.xs.shape))
        print('Shape of {} data y is {}'.format(mode, self.ys.shape))

        self.dim = self.xs.shape[-1]

    def __getitem__(self, index):
        return self.xs[index], self.ys[index]

    def __len__(self):
        return len(self.xs)

class test_dataset__mango(Dataset):
    def __init__(self, test_x):
        self.xs = torch.FloatTensor(test_x)

        print('Shape of test data x is {}'.format(self.xs.shape))

    def __getitem__(self, index):
        return self.xs[index]

    def __len__(self):
        return len(self.xs)

In [ ]:
# Split data into different dataset
train_dataset = train_dataset_mango(train_x, train_y, mode='train')
dev_dataset = train_dataset_mango(train_x, train_y, mode='dev')
test_dataset = test_dataset__mango(test_x)

Shape of train data x is torch.Size([4480, 224, 224, 3])
Shape of train data y is torch.Size([4480, 3])
Shape of dev data x is torch.Size([1120, 224, 224, 3])
Shape of dev data y is torch.Size([1120, 3])
Shape of test data x is torch.Size([800, 224, 224, 3])


## **Config**

In [ ]:
# Set config
config = {
    'batch': 100,
    'Dropout': 0.25,
    'learning_rate': 0.001,
    'weight_decay': 0.02, 
    'epochs': 100, 
    'n_dim': [32, 32, 32, 64],
    'fc_n_dim': [64*26*26, 1024, 256, 128, 64],
    'early_stop': 10,
    'path': './models/cnn.pth'
}

## **Data Loader**

In [ ]:
# Fit data loader
train_loader = DataLoader(train_dataset, batch_size=config['batch'], shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=config['batch'], shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=config['batch'], shuffle=False)

## **Normalization and Build Model**

In [ ]:
# Define CNN with YOLO model
class CNN_YOLO(nn.Module):
    def __init__(self, input_dim):
        super(CNN_YOLO, self).__init__()
        
        # CNN
        self.cnn = nn.Sequential(
            # 3*224*224
            nn.Conv2d(input_dim, config['n_dim'][0], 3, 1),
            nn.BatchNorm2d(config['n_dim'][0]),
            nn.ReLU(),
            # 32*222*222
            nn.Conv2d(config['n_dim'][0], config['n_dim'][1], 3, 1),
            nn.BatchNorm2d(config['n_dim'][1]),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),
            # 32*110*110
            nn.Conv2d(config['n_dim'][1], config['n_dim'][2], 3, 1),  
            nn.BatchNorm2d(config['n_dim'][2]),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),
            # 32*54*54
            nn.Conv2d(config['n_dim'][2], config['n_dim'][3], 3, 1),  
            nn.BatchNorm2d(config['n_dim'][3]),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0)
            # 64*26*26
        )

        # Fully Connected Layer
        self.fc = nn.Sequential(
            nn.Linear(config['fc_n_dim'][0], config['fc_n_dim'][1]),
            nn.Dropout(config['Dropout']),
            nn.ReLU(),
            nn.Linear(config['fc_n_dim'][1], config['fc_n_dim'][2]),        
            nn.Dropout(config['Dropout']),         
            nn.ReLU(),            
            nn.Linear(config['fc_n_dim'][2], config['fc_n_dim'][3]),
            nn.Dropout(config['Dropout']),
            nn.ReLU(),
            nn.Linear(config['fc_n_dim'][3], config['fc_n_dim'][4]),
            nn.Dropout(config['Dropout']),
            nn.ReLU(),
            nn.Linear(config['fc_n_dim'][4], 3)
        )

    def forward(self, x):
        # Change dimension from [data_size, 224, 224, 3] to [data_size, 3, 224, 224]
        x = x.permute(0, 3, 1, 2)
        x = self.cnn(x)

        # Flatten
        x = x.flatten(1)
        x = self.fc(x)
        
        return x

In [ ]:
# Initialize weight
def weight_initialization(layer):
    if isinstance(layer, nn.Linear):
      torch.nn.init.xavier_uniform_(layer.weight)

In [ ]:
# Build model
loss_function = nn.CrossEntropyLoss()
CNN_YOLO_model = CNN_YOLO(train_dataset.dim).to(device)
CNN_YOLO_model.apply(weight_initialization)
optimizer = optim.Adam(CNN_YOLO_model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])

## **Re-load best weight, and prediction test dataset**

In [ ]:
# Define training
def train(train_loader, dev_loader, model, config, device):
    # settings
    epochs = config['epochs']
    path = config['path']
    
    epoch = 0
    min = sys.maxsize
    early_stop_cnt = 0
    
    while epoch < epochs:
        model.train()
        for i, (x,y) in enumerate(train_loader):
            optimizer.zero_grad()
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss = loss_function(pred, y)
            loss.backward()
            optimizer.step()

        if dev_loader is not None:
            dev_loss = dev(dev_loader, model, device)
            if dev_loss < min:
                torch.save(model, path)
                min = dev_loss
                print('epoch {}: loss = {}'.format(epoch+1, min))
                early_stop_cnt = 0
            else:
                early_stop_cnt += 1
        epoch += 1
        if early_stop_cnt > config['early_stop']:
            break

    print('Finished training after {} epochs'.format(epoch))

# Define validation
def dev(dev_loader, model, device):
    model.eval()
    total_loss = 0
    for x, y in dev_loader:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            pred = model(x)
            ce_loss = loss_function(pred, y)
        total_loss += ce_loss.detach().cpu().item()*len(x)
    
    return total_loss / len(dev_loader.dataset)

In [ ]:
# Training
train(train_loader, dev_loader, CNN_YOLO_model, config, device)

epoch 1: loss = 1.037109179156167
epoch 2: loss = 0.9653753223163741
epoch 3: loss = 0.9448735117912292
epoch 4: loss = 0.9363183059862682
epoch 5: loss = 0.910911497260843
epoch 7: loss = 0.8599807481680598
epoch 10: loss = 0.7430454151971
epoch 11: loss = 0.7278367429971695
epoch 12: loss = 0.695417291351727
epoch 16: loss = 0.6681674016373498
epoch 25: loss = 0.6511758661695889
epoch 27: loss = 0.6407711952924728
epoch 28: loss = 0.6318994845662799
epoch 32: loss = 0.6229254720466477
epoch 35: loss = 0.6192143846835408
epoch 40: loss = 0.615894231413092
Finished training after 51 epochs


In [ ]:
# Delete the model
del CNN_YOLO_model

In [ ]:
# Define testing
def test(test_loader, model, device):
    model.eval()
    pred_list = []
    for x in test_loader:
        x = x.to(device)
        with torch.no_grad():
            pred = model(x)
            pred_list.append(pred.detach().cpu())

    pred_list = torch.cat(pred_list, dim=0).numpy()
    print("Done testing")
    return pred_list

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference

In [ ]:
# Testing
CNN_YOLO_model = torch.load(config['path']).to(device)
test_pred = test(test_loader, CNN_YOLO_model, device)
res_list = []
for test in test_pred:
    res = softmax(test)
    x = np.zeros(3)
    x[np.argmax(res)] = 1
    res_list.append(x)

Done testing


In [ ]:
total = 0
for i in range(len(res_list)):
    if np.argmax(res_list[i]) == np.argmax(test_y[i]):
        total += 1
print('accuracy', total/len(res_list))

accuracy 0.76125


In [ ]:
print(type(res_list))

## **Arch**

In [ ]:
summary(CNN_YOLO_model, (224, 224, 3))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 222, 222]             896
       BatchNorm2d-2         [-1, 32, 222, 222]              64
              ReLU-3         [-1, 32, 222, 222]               0
            Conv2d-4         [-1, 32, 220, 220]           9,248
       BatchNorm2d-5         [-1, 32, 220, 220]              64
              ReLU-6         [-1, 32, 220, 220]               0
         MaxPool2d-7         [-1, 32, 110, 110]               0
            Conv2d-8         [-1, 32, 108, 108]           9,248
       BatchNorm2d-9         [-1, 32, 108, 108]              64
             ReLU-10         [-1, 32, 108, 108]               0
        MaxPool2d-11           [-1, 32, 54, 54]               0
           Conv2d-12           [-1, 64, 52, 52]          18,496
      BatchNorm2d-13           [-1, 64, 52, 52]             128
             ReLU-14           [-1, 64,

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(test_y, res_list, average='micro')

0.76125

In [ ]:
f1_score(test_y, res_list, average='macro')

0.7661370820340125

In [ ]:
f1_score(test_y, res_list, average=None)

array([0.77238806, 0.68610635, 0.83991684])